In [1]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
import sys
sys.path.append("/richmondvol1/rusty/stompy")
from stompy import utils
import stompy.model.delft.dflow_model as dfm
import stompy.model.delft.waq_scenario as dwaq
from stompy.grid import unstructured_grid
from matplotlib import colors
import os
import numpy as np
import xarray as xr
from stompy.plot import mkanim

from matplotlib import colors

from stompy import xr_transect
import subprocess
from shapely import wkt

from IPython.display import Video
import six
import glob
from stompy.plot import nbviz, plot_wkb

In [3]:
# initial swimming: Oakland poly release, 8 days, 0, 10, diurnal 10 swimming.
# run_dir="run_wy2022_bloom_common_20220808-v001"
# 0, 0.2/-10, and 0/-10. Also add VSedIM* to output
# run_dir="run_wy2022_bloom_common_20220808-v003"

# run_dir='run_wy2022_bloom_common_20220801'

# neutral, -10/10, -10/0
# run_dir='run_wy2022_bloom_common_20220808-v005'

# same, but longer run
run_dir='run_wy2022_take2_common_20220801'

In [4]:
ds=xr.open_dataset(os.path.join(run_dir,"dwaq_map.nc"))

In [6]:
ds

<xarray.Dataset>
Dimensions:     (sub: 8, time: 1441, layer: 10, face: 49996, node: 44184,
                 maxnode_per_face: 6, edge: 94194, node_per_edge: 2)
Coordinates:
  * sub         (sub) object 'IM1' 'IM2' 'IM3' ... 'VSedIM2' 'VSedIM3'
  * time        (time) datetime64[ns] 2022-08-01 ... 2022-08-31
Dimensions without coordinates: layer, face, node, maxnode_per_face, edge,
                                node_per_edge
Data variables: (12/17)
    t_sec       (time) int32 7948800 7950600 7952400 ... 10539000 10540800
    IM1         (time, layer, face) float32 ...
    IM2         (time, layer, face) float32 ...
    IM3         (time, layer, face) float32 ...
    LocalDepth  (time, layer, face) float32 ...
    SURF        (time, layer, face) float32 ...
    ...          ...
    node_y      (node) float64 4.151e+06 4.152e+06 ... 4.153e+06 4.153e+06
    face_node   (face, maxnode_per_face) int32 ...
    edge_node   (edge, node_per_edge) int32 ...
    bedlevel    (face) float32 -0.0001 -0.0001 -0.0001 ... -0.6486 -0.6485
    eta         (time, face) float32 ...
    sigma       (layer) float64 0.95 0.85 0.75 0.65 0.55 ... 0.35 0.25 0.15 0.05
Attributes:
    header:   DWAQ                                    n/a                    ...

In [7]:
six.moves.reload_module(unstructured_grid)
g=unstructured_grid.UnstructuredGrid.from_ugrid(ds)

In [8]:
zoom=(525166.5357195626, 595636.7532705697, 4147408.0927562183, 4202218.261962557)

In [16]:
# line from W span of bay bridge to about Redwood City
profile="""
LineString (553934.46602910873480141 4185292.98902965523302555, 556003.0745520245982334 4182475.40155878709629178, 557144.37580604711547494 4180335.46170749515295029, 557964.68608237581793219 4178231.18752039084210992, 558250.01139588153455406 4176198.24466166319325566, 558606.66803776356391609 4172631.67824284266680479, 559248.64999315119348466 4169279.10580915166065097, 560211.62292623275425285 4166283.19001734210178256, 561994.90613564301747829 4163750.92785997968167067, 563706.85801667685154825 4161896.31332219298928976, 565882.46353215735871345 4159970.36745603010058403, 568129.40037601429503411 4158650.73788106627762318, 571089.65050363529007882 4156831.78900746814906597, 573728.90965356235392392 4154406.52384267002344131)
"""
profile_geom_channel=wkt.loads(profile) 


profile_shoal="""
LineString (553943.43526522838510573 4185306.34642233327031136, 555905.91317636473104358 4182700.43313049618154764, 557064.09686162555590272 4180319.72222190443426371, 557932.7346255712909624 4178099.8701584879308939, 558318.79585399152711034 4175686.98748086113482714, 558447.48293013160582632 4174046.2272600750438869, 558994.40300372685305774 4171392.05631468491628766, 560184.75845802284311503 4169477.83605710230767727, 562565.46936661447398365 4168030.10645052650943398, 570930.12931572052184492 4165456.3649277244694531, 574211.64975729293655604 4164716.4142399188131094)
"""
# similar northern run, but in the middle of San Bruno Shoal head out across the shoal
profile_geom_shoal=wkt.loads(profile_shoal)


if 0:
    profile_name="channel"
    profile_geom=profile_geom_channel
    
if 1:
    profile_name='shoal'
    profile_geom=profile_geom_shoal


In [17]:
# Grab output at single cell, see how vertical distribution is changing
tracers=['IM1','IM2','IM3']
tracer_names=['0','{-10,10}','{0,10}']

cells,dists=g.select_cells_intersecting(profile_geom,order=True,return_distance=True)
cc=g.cells_center()

c_ref=cells[ len(cells)//2]
depth_ref=ds['TotalDepth'].isel(layer=0,face=c_ref)

In [18]:
def extract_transect(tidx):
    snap=ds.isel(time=tidx)
    tran=xr.Dataset()
    tran['d_sample']=('sample',),dists
    tran['cell']=('sample',),cells
    tran['x_sample']=cc[cells,0]
    tran['y_sample']=cc[cells,1]

    # LocalDepth is bottom of the segments
    depths=-snap['LocalDepth'].isel(face=cells).transpose('face','layer').values
    #tran['z_ctr']=('sample','layer'),depths
    interfaces=np.concatenate( [0*depths[:,:1], depths], axis=1)
    tran['z_int']=('sample','interface'),interfaces
    tran['z_ctr']=('sample','layer'), 0.5*(interfaces[:,:-1] + interfaces[:,1:])

    for tracer in tracers: # + ['VSedIM2','VSedIM3']:
        conc=snap[tracer].isel(face=cells)
        tran[tracer]=('sample','layer'),conc.transpose('face','layer').values
    return tran

In [19]:
save=True
if save:
    frame_dir=os.path.join(run_dir,f'swim_frames_{profile_name}_v0')
    if not os.path.exists(frame_dir):
        os.makedirs(frame_dir)
    [os.unlink(f) for f in glob.glob(os.path.join(frame_dir,'*.png'))]

    patt=os.path.join(frame_dir,'frame_%04d.png')

    plt.ioff()
    tidxs=range(ds.dims['time'])
else:
    tidxs=[-1]

for tidx in tidxs:
    print(tidx)
    tran=extract_transect(tidx)

    fig,axs=plt.subplots(len(tracers)+2,1,figsize=(8.5,7))
    ax_conc=axs[-2]
    ax_t=axs[-1]
    ax_d=ax_t.twinx()

    ccolls=[]
    for tracer,name,ax in zip(tracers,tracer_names,axs):
        ccoll=xr_transect.plot_scalar(tran, tracer, ax=ax,cmap='turbo')
        ccolls.append(ccoll)
        ax_conc.plot(tran.d_sample, tran[tracer].mean(dim='layer'),label=name)

    if 1:
        for coll in ccolls[:-1]:
            coll.set_norm( ccolls[-1].norm )
        
    ax_d.plot(ds.time, depth_ref)
    ax_d.axvline(ds.time.values[tidx],color='k')

    dnum_pdt=utils.to_dnum( ds.time.values-np.timedelta64(7,'h'))
    daylight=(-np.cos(dnum_pdt*2*np.pi)).clip(0)

    ax_t.fill_between(ds.time,daylight,zorder=-1,color='yellow')
    ax_t.axis(ymin=0)
    ax_t.yaxis.set_visible(0)
    
    ax_conc.legend(loc='upper left',bbox_to_anchor=[1.,1],frameon=0,title='w$_s$ (m/d)')
    ax_conc.text(0.01,0.98,"Depth avg.",transform=ax_conc.transAxes, va='top')
    
    fig.autofmt_xdate()
    fig.subplots_adjust(top=0.98,right=0.85,left=0.06,bottom=0.12)
    
    cax=fig.add_axes( [0.89, axs[2].get_position().ymin, 0.02,
                       axs[0].get_position().ymax - axs[2].get_position().ymin] )
    plt.colorbar(ccolls[-1], cax=cax)

    
    if save:
        fig.savefig(patt%tidx)
        plt.close(fig)

if save:
    plt.ion()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [20]:
swim_mp4=os.path.join(run_dir,f'swim_surfing_{profile_name}_v0.mp4')
mkanim.mkanim(swim_mp4,patt,fps=12)

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [21]:
Video(swim_mp4)

In [22]:
save_output=True

if save_output:
    tidxs=range(ds.dims['time'])
    plt.ioff()
    
    frame_dir=os.path.join(run_dir,'swim_map_v1')
    patt=os.path.join(frame_dir,'frame_%04d.png')
    if not os.path.exists(frame_dir):
        os.makedirs(frame_dir)
        for fn in glob.glob(os.path.join(frame_dir,'*.png')):
            os.unlink(fn) 
else:
    tidxs=[30]

for tidx in tidxs:
    fig,axs=plt.subplots(1,3,figsize=(8.5,4))
    for ax in axs:
        ax.axis('off')
    fig.subplots_adjust(0,0,1,1,hspace=0.01,wspace=0.01)

    cax=fig.add_axes([0.3,0.09,0.4,0.02])

    layer=0
    layer_name="Surface"

    clim=[0,0.5]

    for ax,tracer,name in zip(axs,tracers,tracer_names):
        scal=ds[tracer].isel(time=tidx,layer=layer).values

        ccoll=g.plot_cells(values=scal,cmap='turbo',lw=0.5,edgecolor='face',
                           ax=ax)
        ccoll.set_clim(clim)
        for prof_geom in [profile_geom_channel,profile_geom_shoal]:
            plot_wkb.plot_wkb(prof_geom,ax=ax,color='r',ls='--')
        ax.axis('equal')
        ax.axis(zoom)
        ax.text(#0.03,0.99,
                0.5,0.77,
                f'w$_s$={name}',transform=ax.transAxes,va='top')

    time_str=ds.time.isel(time=tidx).dt.strftime('%Y-%m-%d %H:%M').item()
    fig.text(0.04,0.07,f'{layer_name}  {time_str}')

    plt.colorbar(ccoll,cax=cax,orientation='horizontal')
    if save_output:
        fig.savefig(patt%tidx)
        plt.close(fig)
        
if save_output:
    plt.ion()

In [23]:
swim_mp4=os.path.join(run_dir,'swim_map_v0.mp4')
mkanim.mkanim(swim_mp4,patt,fps=12)

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [24]:
Video(swim_mp4)

In [184]:
# Checking for mass conservation
if 0:
    for tidx in range(0,ds.dims['time'],20):
        print(f'tidx={tidx} time={ds.time.values[tidx]}')
        snap=ds.isel(time=tidx)

        loc_depth=snap['LocalDepth'].values
        dz=np.diff( np.concatenate([0*loc_depth[:1,:],loc_depth]), axis=0)

        V=g.cells_area()[None,:] * dz
        for tracer in tracers:
            mass=np.nansum( (snap[tracer].values*V) )
            print(f'{tracer}: {mass}')
        print()

tidx=0 time=2022-08-08T00:00:00.000000000
IM1: 1105904362.11729
IM2: 1105904362.11729
IM3: 1105904362.11729

tidx=20 time=2022-08-08T10:00:00.000000000
IM1: 1105904272.5352879
IM2: 1105904270.905971
IM3: 1105904264.057618

tidx=40 time=2022-08-08T20:00:00.000000000
IM1: 1105770129.70497
IM2: 1105800229.4389987
IM3: 1105767927.1901972

tidx=60 time=2022-08-09T06:00:00.000000000
IM1: 1105132923.1545577
IM2: 1105252979.5494313
IM3: 1105128632.0047138

tidx=80 time=2022-08-09T16:00:00.000000000
IM1: 1104640504.098006
IM2: 1104825445.0697107
IM3: 1104630042.4690819

tidx=100 time=2022-08-10T02:00:00.000000000
IM1: 1098671885.540763
IM2: 1099602560.126976
IM3: 1098469746.7057774

tidx=120 time=2022-08-10T12:00:00.000000000
IM1: 1092306480.5443892
IM2: 1093847918.1521306
IM3: 1092053932.918041

tidx=140 time=2022-08-10T22:00:00.000000000
IM1: 1082512901.6999686
IM2: 1085269792.183079
IM3: 1081961505.129893

tidx=160 time=2022-08-11T08:00:00.000000000
IM1: 1066201206.8738737
IM2: 1069492581.52